In [1]:
import pandas as pd
import numpy as np
import requests
import grequests
import os.path
import tensorflow as tf

In [ ]:
# shamelessly taken from https://danijar.com/variable-sequence-lengths-in-tensorflow/
def getSequenceLengths(batchOfSequences):
  used = tf.sign(tf.reduce_max(tf.abs(batchOfSequences), 2))
  length = tf.reduce_sum(used, 1)
  length = tf.cast(length, tf.int32)
  return length

In [ ]:
flags = tf.flags
logging = tf.logging

In [ ]:
flags.DEFINE_bool("train", True, "train mode (test mode if not)")
FLAGS = flags.FLAGS

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)

In [3]:
df = pd.read_pickle('final_processed_invoker_data')

In [12]:
columnDict = {}
for i, v in enumerate(df.columns.values):
    columnDict[i] = v
indexDict = {v: k for k, v in columnDict.items()}
numberEntries = len(df)
batchSize = int(numberEntries/20)

In [ ]:
# 265 is highest ID, and we need a zero for empty
DIM = 266
matchItemsTensor = tf.placeholder(tf.float32, shape=shape=[None, None, DIM])
batched_ItemData = tf.train.batch(
    tensors=[matchItemsTensor],
    batch_size=batchSize,
    dynamic_pad=True,
    name="lstm_batch"
)

# shift each sequence to the left by one (stride to the right by 1)
# and then pad at the end to maintain same lengths!
batched_ItemData_strided = tf.strided_slice(batched_data, [0, 1, 0], batched_data.shape)
targets = tf.pad(batched_ItemData_strided, [[0, 0], [0, 1], [0, 0]], "CONSTANT")

batched_ItemDataSequenceLengths = getSequenceLengths(batched_ItemData)
lstmHiddenSize = 50
lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmHiddenSize)

In [ ]:
lstmOutput, last_states = tf.nn.dynamic_rnn(
    cell=lstmCell,
    dtype=tf.float64,
    sequence_length=batchSequenceLengths,
    inputs=batched_data)

In [ ]:
lstmOutput = tf.reshape(lstmOutput, [-1, lstmHiddenSize])

In [ ]:
numberOfReps = lstmOutput.shape[0]
numberOfFeaturesInHeroInfoMap = heroInfoMap.shape[1]
heroInfoMap = tf.tile(heroInfoMap, [numberOfReps])
heroInfoMap = tf.reshape(heroInfoMap, [-1, numberOfFeaturesInHeroInfoMap])
comboLayer = tf.concat([lstmOutput, heroInfoMap], 1)
logits = tf.matmul(comboLayer, weight) + bias


loss = tf.nn.softmax_cross_entropy_with_logits(labels = targets, logits = logits)

In [ ]:
# default of 0.001
learning_rate = 0.001
# default of 0.99
beta1 = 0.99
# default of 0.999
beta2 = 0.999
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate, beta1 = beta1, beta2 = beta2)
train_op = optimizer.minimize(loss)

In [ ]:
variable_dict = {'weights_1': weights_1, 'weights_2': weights_2,
                     'bias_1': bias_1, 'bias_2': bias_2}
saver = tf.train.Saver(variable_dict)
init = tf.global_variables_initializer()